<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/RBM_Gibbs_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import WandB 

In [1]:
!pip install wandb
import wandb


     |████████████████████████████████| 1.8MB 3.9MB/s 
     |████████████████████████████████| 174kB 45.7MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 133kB 36.0MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=6afcbe7702c1a8987becb74eb61996ed1344eb68968a1c67ba97641c4e8962fc
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=6d55e8675b8a36bd6c9111653581a9a7bd7b6e6d752d02faa3c4eb50f297bd87
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.manifold import TSNE

**Read the Fashion MNIST data set, divide the data set as train and validation. There are 10 class in the data set. Initialize all the variables e.g : number of hidden layer size, number of class etc**

In [3]:
# load the Fashion MNIST data set
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_type = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 

Project='ASSIGNMENT 4'


# Split the data set and assign the data values depending on thresholding
X_train = np.array(x_train.reshape(x_train.shape[0], 784,1))         
X_test  = np.array(x_test.reshape(x_test.shape[0], 784,1))           
X_train = (X_train > 126) * 1                                        
X_test  = (X_test > 126) * 1                                         
X_val = X_train[-15000:]                                             
X_train = X_train[0:45000]                                           

Y_train = np.zeros([len(y_train),10,1])
Y_test = np.zeros([len(y_test),10,1])

for i in range(len(y_train)):                                       
  y = np.zeros([10, 1])
  y[y_train[i]] = 1.0
  Y_train[i] = y

for i in range(len(y_test)):                                         
  y = np.zeros([10, 1])
  y[y_test[i]] = 1.0
  Y_test[i] = y                                                      

Y_val = Y_train[-15000:]                                             
Y_train = Y_train[0:45000]        


n_vis = X_train.shape[1]                                         
n_train = X_train.shape[0]                                  
n_val = X_val.shape[0]                                      
n_test = X_test.shape[0]                                    
n_hidden = 64                 
n_class = 10


4431872/4422102 [==============================] - 0s 0us/step


**Define activation functions, Initalize Parameters**

In [4]:
def sigmoid(x) :                                                                              
	
  return 1.0/(1.0+np.exp(-x))

def softmax(x):                                                                               
  return np.exp(x) / np.sum(np.exp(x))  


def RBM_init_params() :                                                                        
	
  RBM_params = {}
  RBM_params["W"] = np.random.randn(n_hidden, n_vis)*np.sqrt(6./(n_vis + n_hidden))   
  RBM_params["h_b"] = np.zeros((n_hidden,1),dtype=np.float64)                                                     
  RBM_params["v_b"] = np.zeros((n_vis,1),dtype=np.float64)

  return RBM_params

def Class_params_init() :                                                                 

  class_params = {}
  class_params["W"] = np.random.randn(n_class, n_hidden)*np.sqrt(6./(n_class + n_hidden))       
  class_params["b"] = np.zeros((n_class,1),dtype=np.float64)

  return class_params


def hidden_representation(x,parameters) :                                                               
   
    W = parameters["W"]
    h_b = parameters["h_b"]
    hidden_prob = sigmoid(np.dot(W,x)+h_b)
    hidden_rep = np.random.binomial(1,hidden_prob)
    
    return hidden_rep                                   

# **Train the RBM model using Gibbs sampling and Train the classifer**

In [11]:
def Train_RBM(X_train,parameters,k,learning_rate,r) :                       # Function to train the RBM
	
  W   = parameters["W"]
  h_b = parameters["h_b"]
  v_b = parameters["v_b"]
  
  for i in range(n_train) :
    
      v_s    = np.random.rand(784,1)
      v_in   = X_train[i]
      dw     = np.zeros(np.shape(W))
      dv     = np.zeros(np.shape(v_b))
      dh     = np.zeros(np.shape(h_b))
                          
      for t in range(k + r):
         if t < k:
                                                                                            
            p_h_g_v = sigmoid(np.dot(W,v_s)+h_b)                                  
            h_s = np.random.binomial(1,p_h_g_v)                                      
            p_v_g_h = sigmoid(np.dot(np.transpose(W),h_s)+v_b)                    
            v_s = np.random.binomial(1,p_v_g_h)                                      
         else:
            
            p_h_g_v = sigmoid(np.dot(W,v_s)+h_b)                                  
            h_s = np.random.binomial(1,p_h_g_v)                                     
            p_v_g_h = sigmoid(np.dot(np.transpose(W),h_s)+v_b)                    
            v_s = np.random.binomial(1,p_v_g_h)    
            
            dw = dw + 1/r*(np.dot(sigmoid(np.dot(W,v_s)+h_b),np.transpose(v_s)))
            dv = dv + 1/r*(v_s)
            dh = dh + 1/r*(sigmoid(np.dot(W,v_s)+h_b))

      # Update Rule
      W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_in)+h_b),np.transpose(v_in)) - dw)
      v_b = v_b + learning_rate*(v_in-dv)
      h_b = h_b + learning_rate*(sigmoid(np.dot(W,v_in)+h_b) - dh)

  parameters["W"] = W
  parameters["h_b"] = h_b
  parameters["v_b"] = v_b

  return parameters

def Train_class(X,Y,RBM_params,param_cls,eph_c,learning_rate)  :                                  
    
    W = param_cls["W"]
    b = param_cls["b"]
    
    for epoch in range(eph_c) :
      for i in range(n_val) :
         # feed forward
         hidden_rep = hidden_representation(X[i],RBM_params)                                         
         previous_out = np.dot(W,hidden_rep)+b
         y_hat = softmax(previous_out)
         # backpropogate
         dW = np.dot(-(Y[i]-y_hat),np.transpose(hidden_rep))
         db = -(Y[i]-y_hat)
         # Update Classifier weights
         W = W - learning_rate*dW
         b = b - learning_rate*db

    param_cls["W"] = W
    param_cls["b"] = b 
    
    return param_cls   


# **RBM classifier**



In [12]:

def RBM_clsfier(X_train,Y_train,X_val,Y_val,X_test,Y_test,epochs_RBM,eph_c,k,n_hidden,learning_rate,r) :
  param_cls  = Class_params_init() 
  RBM_params = RBM_init_params()

  for j in range(epochs_RBM) :
     RBM_params = Train_RBM(X_train,RBM_params,k,learning_rate,r)                             
     param_cls = Train_class(X_val,Y_val,RBM_params,param_cls,eph_c,learning_rate)               
     
     accuracy = 0.0
     loss = 0.0 
     # Evaluate accuracy and loss over test data
     for i in range(n_test) :
        h = hidden_representation(X_test[i],RBM_params)
        y_hat = softmax(np.dot(param_cls["W"],h)+param_cls["b"])

        if y_hat.argmax()==Y_test[i].argmax():
            accuracy = accuracy + 1
        loss = loss + -1*np.sum(np.multiply(y,np.log(y_hat)))

     accuracy = accuracy/n_test
     loss = loss/n_test 
     print("Epoch :" + str(j)+" "+ str(accuracy)+" "+str(loss))
     wandb.log({"Accuracy":accuracy,"Loss":loss,"Epoch":j})
  return RBM_params,param_cls, accuracy, loss



In [13]:
def train():
  hyperparameter_defaults=dict(
        epochs_RBM = 5,
        eph_c = 3,
        k = 40,
        n_hidden = 80,
        learning_rate = 0.1 ,
        r = 10                                                                   
        )

  wandb.init(config=hyperparameter_defaults)

  config=wandb.config
  RBM_params,param_cls,accuracy,loss = RBM_clsfier(X_train,Y_train,X_val,Y_val,X_test,Y_test,config.epochs_RBM,config.eph_c,config.k,config.n_hidden,config.learning_rate,config.r)



**Sweep over all the hyperparameters**

In [ ]:
def sweeper(sweep_config,Project):
  sweep_id=wandb.sweep(sweep_config,project=Project,entity='chaxin',)
  sweep_id = "nhvl1lun"
  wandb.agent(sweep_id,train,project=Project,entity='chaxin',)


#sweep dictionary
sweep_config={
    'method':'bayes',
    'metric':{
        'name':'accuracy',
        'goal':'maximize'},

}

parameters_sweep={
    
    'epochs_RBM':{
      'values':[7]  
    },
    'eph_c':{
        'values':[1]
    },
    'k':{
        'values':[1,5,10,15,20]
    },
    'n_hidden':{
        'values':[64,128,256]
    },
    'learning_rate':{
        'values':[0.01]
    },
    'r':{
        'values' :[10,20]
    }
    
}

sweep_config['parameters']=parameters_sweep  

In [14]:
train()

Epoch :0 0.6634 9.465510759889954


KeyboardInterrupt: ignored